In [1]:
from indo_eval.generate_sql_templates import generate_sql_templates, save_sql_templates_to_text, generate_prompt
from indo_eval.db_tool import db_tool

In [6]:
# one table
one_placeholder = """- Each query must contain at least one parameter placeholder in the WHERE clause. \
    \n\t"""
no_answer_multiplicity = """- Ensure the query yields a specific and singular answer to avoid multiplicity issues. "SELECT Name, StartDate, EndDate FROM Project WHERE StartDate >= '[Project.StartDate]'" is an invalid example since mulitple projects can start after '[Project.StartDate]' \
    \n\t"""
one_perspective = """- Select only one specific column \
    \n\t"""
one_table = """- Each query must involve only one entity (table). \
    \n\t"""
entity_types = db_tool.infer_entity_types_from_table_names(verbose=False) #['Project', 'Client', 'Employee']
entity_types = [entity_types[1]]
constraint1 = one_perspective+no_answer_multiplicity+one_placeholder
# Join Two Entities/Tables
# no_unexplainable_columns = """- MUST NOT use unexplainable columns (e.g., ID) in the WHERE clause. An INVALID example: SELECT Company.Name FROM Company INNER JOIN Company_Project ON Company.CompanyID = Company_Project.CompanyID INNER JOIN Project ON Project.ProjectID = Company_Project.ProjectID WHERE Project.ID = [Project.ID]; A VALID example: SELECT Company.Name FROM Company INNER JOIN Company_Project ON Company.CompanyID = Company_Project.CompanyID INNER JOIN Project ON Project.ProjectID = Company_Project.ProjectID WHERE Project.Name = [Project.Name]\
#     \n\t"""
# # SELECT Project.Name FROM Project INNER JOIN Company_Project ON Project.ProjectID = Company_Project.ProjectID WHERE Company_Project.CompanyID = '[Company.CompanyID]'; 
# # SELECT Project.Name FROM Project INNER JOIN Company_Project ON Project.ProjectID = Company_Project.ProjectID WHERE Company.CompanyName = '[Company.CompanyName]' 
# two_tables = """- Each query must involve the relation betwen {} and {}. \
#     \n\t"""
# entity_types = ["Employee", "Project"] 
# two_tables = two_tables.format(entity_types[0], entity_types[1])
# constraint2 = two_tables+no_unexplainable_columns
# print(generate_prompt(entity_types, constraint1)[0])
# print(generate_prompt(entity_types, constraint1)[1])

templates = generate_sql_templates(entity_types, constraint1)
print(templates)



 
    ##DATA SCHEMA##
    Table: Employee
	 Column: Name (VARCHAR(255))
	 Column: JobTitle (VARCHAR(255))
	 Column: Department (VARCHAR(255))
	 Column: SupervisorOrManager (VARCHAR(255))
 

    ##RESPONSE_START##
    
You are a SQL query Template Generator: Generate ACCEPTABLE SQL query templates with placeholders according to the give data schema and requirements.
    A simple example of an acceptable SQL query template is: SELECT Industry FROM Company WHERE Name = '[Company.Name]';
    
    You must follow the basic criteria below: 
    ##CRITERIA##     
	- The placeholder format should be a combination of a table name and a column name, enclosed within square brackets, e.g., '[User.Name]'.    
	- Use only 'SELECT' queries.    
	- Select specific column(s) instead of using '*'. Avoid projecting attributes that appear in the predicate.    
	- The selected and condition columns in the query MUST BE MEANINGFUL and DESCRIPTIVE to ensure the queries are easily understood by non-technical 

In [7]:
# save to text
save_sql_templates_to_text(templates, f"{'_'.join(entity_types)}.txt")